In [100]:
import pandas as pd
from tqdm import tqdm

In [101]:
promo_combined = pd.read_csv("D:\\Competitions\\DataStorm-2022\\semi-final\\data\\merged_promo_item_df.csv")
promo_data = pd.read_csv("D:\\Competitions\\DataStorm-2022\\semi-final\\data\\promotion_data.csv")
train_data = pd.read_csv("D:\\Competitions\\DataStorm-2022\\semi-final\\data\\train_data.csv")
test_data = pd.read_csv("D:\\Competitions\\DataStorm-2022\\semi-final\\data\\test_data.csv")
val_data = pd.read_csv("D:\\Competitions\\DataStorm-2022\\semi-final\\data\\validation_data.csv")

In [102]:
promo_data

,ItemCode,PromotionStartDate,PromotionEndDate,DiscountValue,DiscountType,SellingPrice
0,379249,12/3/2021,12/9/2021,10,Percentage,70
1,1066570,11/19/2021,11/25/2021,15,Percentage,45
2,124954,10/29/2021,11/4/2021,15,Percentage,210
3,35530,10/22/2021,10/28/2021,15,Percentage,60
4,1013335,1/21/2022,1/27/2022,10,Percentage,210
...,...,...,...,...,...,...
309,17287,12/10/2021,12/16/2021,10,Percentage,70
310,132028,10/29/2021,11/4/2021,10,Percentage,45
311,1076938,3/11/2022,3/17/2022,10,Percentage,80
312,17287,12/24/2021,12/30/2021,10,Percentage,70


In [103]:
promo_data["PromotionStartDate"] = pd.to_datetime(promo_data["PromotionStartDate"])
promo_data["PromotionEndDate"] = pd.to_datetime(promo_data["PromotionEndDate"])

In [104]:
train_data["ItemCode"].value_counts()

1047967    140
1067092    140
59749      140
17296      140
20824      140
          ... 
1090114     64
1090294     62
1015621     61
52099       59
904984      58
Name: ItemCode, Length: 197, dtype: int64

In [105]:
promo_data["ItemCode"].value_counts()

1066570    14
1085749    13
1048975    13
17287      13
379249     12
           ..
815101      1
1090276     1
687616      1
903976      1
1071124     1
Name: ItemCode, Length: 66, dtype: int64

In [106]:
promo_combined.index = pd.MultiIndex.from_frame(promo_combined[["ItemCode","DateID"]], names=["ItemCode","DateID"])

In [107]:
promo_combined

CategoryCode  ItemCode      DateID  DailySales  \
ItemCode DateID                                                      
379249   2021-12-04   category_1    379249  2021-12-04           9   
         2021-12-07   category_1    379249  2021-12-07          11   
         2021-12-05   category_1    379249  2021-12-05           9   
         2021-12-08   category_1    379249  2021-12-08           9   
         2021-12-06   category_1    379249  2021-12-06           6   
...                          ...       ...         ...         ...   
23569    2021-11-20   category_2     23569  2021-11-20           6   
         2021-11-13   category_2     23569  2021-11-13           3   
         2021-11-15   category_2     23569  2021-11-15           6   
         2021-11-14   category_2     23569  2021-11-14           7   
         2021-11-17   category_2     23569  2021-11-17           8   

                    PromoStarted  PromoEnded  DiscountValue DiscountType  \
ItemCode DateID                                                            
379249   2021-12-04   2021-12-03  2021-12-09             10   Percentage   
         2021-12-07   2021-12-03  2021-12-09             10   Percentage   
         2021-12-05   2021-12-03  2021-12-09             10   Percentage   
         2021-12-08   2021-12-03  2021-12-09             10   Percentage   
         2021-12-06   2021-12-03  2021-12-09             10   Percentage   
...                          ...         ...            ...          ...   
23569    2021-11-20   2021-11-19  2021-11-25             10   Percentage   
         2021-11-13   2021-11-12  2021-11-18             10   Percentage   
         2021-11-15   2021-11-12  2021-11-18             10   Percentage   
         2021-11-14   2021-11-12  2021-11-18             10   Percentage   
         2021-11-17   2021-11-12  2021-11-18             10   Percentage   

                     SellingPrice  
ItemCode DateID                    
379249   2021-12-04            70  
         2021-12-07            70  
         2021-12-05            70  
         2021-12-08            70  
         2021-12-06            70  
...                           ...  
23569    2021-11-20            80  
         2021-11-13            80  
         2021-11-15            80  
         2021-11-14            80  
         2021-11-17            80  

[8912 rows x 9 columns]

In [108]:
train_data

,CategoryCode,ItemCode,DateID,DailySales
0,category_2,1071115,12/19/2021,1
1,category_4,210652,2/13/2022,1
2,category_2,877624,12/19/2021,1
3,category_1,371104,12/19/2021,3
4,category_1,1090294,12/19/2021,3
...,...,...,...,...
20646,category_1,36898,10/12/2021,1
20647,category_2,111436,10/12/2021,1
20648,category_2,815101,10/12/2021,4
20649,category_3,1064572,10/12/2021,1


In [109]:
train_data.insert(2,"SellingPrice",None)
train_data.insert(3,"DistanceToClosestPromo",None)
train_data.insert(4,"onPromo",None)
train_data["DateID"] = pd.to_datetime(train_data["DateID"])

In [110]:
lis = []
for index in tqdm(train_data.index):
    row = train_data.loc[index]
    promotion_per_item_code = promo_data[promo_data["ItemCode"] == row["ItemCode"]]
    promotion_per_item_code.sort_values(by=['PromotionStartDate'])
    if not promotion_per_item_code.shape[0] == 0:
        with_sel_range = promotion_per_item_code[(promotion_per_item_code["PromotionStartDate"]<row["DateID"]) & (promotion_per_item_code["PromotionEndDate"]>row["DateID"])]

        if not with_sel_range.shape[0] == 0:
            train_data.at[index,"SellingPrice"] = with_sel_range.at[with_sel_range.index[0],"SellingPrice"]
            train_data.at[index,"DistanceToClosestPromo"] = 0
        else:
            if promotion_per_item_code.at[promotion_per_item_code.index[0],'PromotionStartDate']>row["DateID"]:
                train_data.at[index,"SellingPrice"] = promotion_per_item_code.at[promotion_per_item_code.index[0],'SellingPrice']
                train_data.at[index,"DistanceToClosestPromo"] = promotion_per_item_code.at[promotion_per_item_code.index[0],'PromotionStartDate']-row["DateID"]
            elif promotion_per_item_code.at[promotion_per_item_code.index[-1],'PromotionEndDate']<row["DateID"]:
                train_data.at[index,"SellingPrice"] = promotion_per_item_code.at[promotion_per_item_code.index[-1],'SellingPrice']
                train_data.at[index,"DistanceToClosestPromo"] = row["DateID"] - promotion_per_item_code.at[promotion_per_item_code.index[0],'PromotionStartDate']
            else:
                for index_0_i in range(len(promotion_per_item_code.index)-1):
                    index_0 = promotion_per_item_code.index[index_0_i]
                    n_index_0 = promotion_per_item_code.index[index_0_i+1]
                    if promotion_per_item_code.at[index_0,"PromotionEndDate"]<row["DateID"]<promotion_per_item_code.at[n_index_0,"PromotionStartDate"]:
                        train_data.at[index,"SellingPrice"] = (promotion_per_item_code.at[index_0,"SellingPrice"]+promotion_per_item_code.at[n_index_0,"SellingPrice"])/2

                        train_data.at[index,"DistanceToClosestPromo"] = min(row["DateID"]-promotion_per_item_code.at[index_0,"PromotionEndDate"],promotion_per_item_code.at[n_index_0,"PromotionStartDate"]-row["DateID"])

                if train_data.at[index, "SellingPrice"] is None:
                    lis.append(index)

                train_data.at[index,"DistanceToClosestPromo"] = -1
    else:
        train_data.at[index,"DistanceToClosestPromo"] = -1




100%|██████████| 20651/20651 [00:23<00:00, 893.59it/s] 


In [111]:
promo_data.loc[promo_data["ItemCode"].isin(list(set(lis)))]

,ItemCode,PromotionStartDate,PromotionEndDate,DiscountValue,DiscountType,SellingPrice


In [112]:
train_data.isnull().sum()

CategoryCode                  0
ItemCode                      0
SellingPrice              13726
DistanceToClosestPromo        0
onPromo                   20651
DateID                        0
DailySales                    0
dtype: int64

In [113]:
train_data

,CategoryCode,ItemCode,SellingPrice,DistanceToClosestPromo,onPromo,DateID,DailySales
0,category_2,1071115,None,-1,None,2021-12-19,1
1,category_4,210652,None,-1,None,2022-02-13,1
2,category_2,877624,None,-1,None,2021-12-19,1
3,category_1,371104,None,-1,None,2021-12-19,3
4,category_1,1090294,None,-1,None,2021-12-19,3
...,...,...,...,...,...,...,...
20646,category_1,36898,None,-1,None,2021-10-12,1
20647,category_2,111436,None,-1,None,2021-10-12,1
20648,category_2,815101,80,59 days 00:00:00,None,2021-10-12,4
20649,category_3,1064572,370,150 days 00:00:00,None,2021-10-12,1


In [114]:
train_data.index = pd.MultiIndex.from_frame(train_data[["ItemCode","DateID"]], names=["ItemCode","DateID"])

In [115]:
train_data.loc[promo_combined.index,"onPromo"] = 1
train_data["onPromo"] = train_data["onPromo"].fillna(0)

In [116]:
train_data.isnull().sum()

CategoryCode                  0
ItemCode                      0
SellingPrice              13726
DistanceToClosestPromo        0
onPromo                       0
DateID                        0
DailySales                    0
dtype: int64

In [117]:
train_data

,,CategoryCode,ItemCode,SellingPrice,DistanceToClosestPromo,onPromo,DateID,DailySales
ItemCode,DateID,,,,,,,
1071115,2021-12-19,category_2,1071115,None,-1,0,2021-12-19,1
210652,2022-02-13,category_4,210652,None,-1,0,2022-02-13,1
877624,2021-12-19,category_2,877624,None,-1,0,2021-12-19,1
371104,2021-12-19,category_1,371104,None,-1,0,2021-12-19,3
1090294,2021-12-19,category_1,1090294,None,-1,0,2021-12-19,3
...,...,...,...,...,...,...,...,...
36898,2021-10-12,category_1,36898,None,-1,0,2021-10-12,1
111436,2021-10-12,category_2,111436,None,-1,0,2021-10-12,1
815101,2021-10-12,category_2,815101,80,59 days 00:00:00,0,2021-10-12,4


In [118]:
train_data.to_csv("editted.csv", index=False)